In [2]:
import yfinance as yf
import pandas as pd

In [ ]:
def calculate_ema(df, span):
    return df['Close'].ewm(span=span, adjust=False).mean()

def calculate_atr(df, period=14):
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift())
    low_close = abs(df['Low'] - df['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    return true_range.rolling(period).mean()

def calculate_rsi(df, period=14):
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

df = yf.download("QQQM", period='100d', auto_adjust=True)

df['EMA10'] = calculate_ema(df, 10)
df['EMA50'] = calculate_ema(df, 50)
df['ATR'] = calculate_atr(df, 14)
df['RSI'] = calculate_rsi(df, 14)
df['Volume_SMA'] = df['Volume'].rolling(20).mean()

df.dropna(inplace=True)

if df.empty:
    print(f"⚠️ Insufficient data to generate signal for {"QQQM"}")

latest = df.iloc[-1:]

try:
    ema10 = float(latest['EMA10'].iloc[0])
    ema50 = float(latest['EMA50'].iloc[0])
    atr = float(latest['ATR'].iloc[0])
    close = float(latest['Close'].iloc[0])
    volume = int(latest['Volume'].iloc[0])
    volume_sma = float(latest['Volume_SMA'].iloc[0])
    rsi = float(latest['RSI'].iloc[0])
    
    print(rsi)
except Exception as e:
    print(f"⚠️ Failed to extract indicators for {"QQQM"}: {e}")

In [17]:
import pandas as pd
from ib_insync import IB, Stock
from datetime import datetime
import nest_asyncio
import asyncio

nest_asyncio.apply()  # ← Required to re-enter the event loop in Jupyter

ib = IB()

# 👇 This must be awaited inside an async def block
async def main():
    await ib.connectAsync('127.0.0.1', 7497, clientId=111)

    WATCHLIST = ['QQQM', 'VOO', 'IAU', 'IEFA', 'MCHI']
    result_dfs = {}

    for symbol in WATCHLIST:
        print(f"→ Fetching {symbol} from IBKR...")
        contract = Stock(symbol, 'SMART', 'USD')
        ib.qualifyContracts(contract)

        bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='100 D',
            barSizeSetting='1 day',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1
        )

        df = pd.DataFrame(bars)
        df.rename(columns={
            'close': 'Close',
            'high': 'High',
            'low': 'Low',
            'volume': 'Volume'
        }, inplace=True)

        print(f"[{symbol}] ✅ {len(df)} rows loaded.")
        result_dfs[symbol] = df
    
    return result_dfs

# Run the async event loop
dfs = asyncio.run(main())

→ Fetching QQQM from IBKR...
[QQQM] ✅ 100 rows loaded.
→ Fetching VOO from IBKR...
[VOO] ✅ 100 rows loaded.
→ Fetching IAU from IBKR...
[IAU] ✅ 100 rows loaded.
→ Fetching IEFA from IBKR...
[IEFA] ✅ 100 rows loaded.
→ Fetching MCHI from IBKR...
[MCHI] ✅ 100 rows loaded.


In [26]:
print(dfs["QQQM"].sort_values(by=["date"], ascending=False))

          date    open    High     Low   Close     Volume  average  barCount
99  2025-04-23  188.64  190.77  186.46  187.07  2744304.0  188.789     12258
98  2025-04-22  180.70  184.27  180.18  183.00  1935101.0  182.336      7013
97  2025-04-21  180.44  180.78  176.19  178.40  1757461.0  177.787      6174
96  2025-04-17  184.12  184.35  181.74  182.86  2357877.0  183.119      9213
95  2025-04-16  184.86  186.34  180.25  182.90  2527429.0  183.160      8881
..         ...     ...     ...     ...     ...        ...      ...       ...
4   2024-12-03  211.54  212.85  211.32  212.75   742986.0  212.175      2614
3   2024-12-02  210.33  212.49  210.23  212.10  1348158.0  211.775      4424
2   2024-11-29  208.24  210.03  207.98  209.80   604074.0  209.239      2270
1   2024-11-27  209.15  209.19  206.61  208.00  1101152.0  207.765      3222
0   2024-11-26  209.12  209.95  208.78  209.63   914191.0  209.448      2766

[100 rows x 8 columns]
